In [1]:
import pandas as pd
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [6]:
f = './Resources/model_data.csv'
final_df = pd.read_csv(f)
final_df.head(5)

,course,band,dist.m.,season,race_group,race_type,Race_Money,trainer,jockey,pos,btn,age,lbs,gear,or,ts,rpr,prob,prize_money
0,Fairyhouse,4yo+,3200,5,Hurdle,Handicap,21690.0,Mervyn Torrens,Mr K H Clarke,4.0,7.00,5,134,t,90.0,89.0,92.0,0.058824,697.5
1,Fairyhouse,4yo+,3200,5,Hurdle,Handicap,21690.0,Noel Meade,Denis O'Regan,13.0,23.75,5,158,b,107.0,86.0,92.0,0.058824,0.0
2,Fairyhouse,4yo+,3200,5,Hurdle,Handicap,21690.0,A L T Moore,Daniel Howard,15.0,25.25,7,146,t,95.0,72.0,79.0,0.076923,0.0
3,Fairyhouse,4yo+,3200,5,Hurdle,Handicap,21690.0,Lester Winters,Mark Walsh,17.0,34.75,4,142,b,94.0,59.0,68.0,0.047619,0.0
4,Fairyhouse,4yo+,3200,5,Hurdle,Handicap,21690.0,H Rogers,M G Cleary,19.0,45.25,6,132,tp,86.0,38.0,50.0,0.047619,0.0


In [8]:
final_df.shape

(126391, 19)

In [9]:
# Take a look at the column variable: prop
print(final_df['prob'].sort_values(ascending=True))

49037     0.003984
102133    0.003984
90567     0.003984
93851     0.003984
94011     0.003984
            ...   
21961     0.952381
103720    0.952381
103762    0.961538
112384    0.961538
112373    0.970874
Name: prob, Length: 126391, dtype: float64


In [10]:
# Assign outcomes as 0 if prob <=0.09 and as 1 if prob >0.09
final_df['prob'] = [0 if x <=0.09 else 1 for x in final_df['prob']]

In [11]:
# Assign X as dataframe of the features and y as a series of the outcome variable
X_data = final_df.drop("prob", 1) 
y = final_df.prob # target

In [12]:
X_data.head(2)

,course,band,dist.m.,season,race_group,race_type,Race_Money,trainer,jockey,pos,btn,age,lbs,gear,or,ts,rpr,prize_money
0,Fairyhouse,4yo+,3200,5,Hurdle,Handicap,21690.0,Mervyn Torrens,Mr K H Clarke,4.0,7.00,5,134,t,90.0,89.0,92.0,697.5
1,Fairyhouse,4yo+,3200,5,Hurdle,Handicap,21690.0,Noel Meade,Denis O'Regan,13.0,23.75,5,158,b,107.0,86.0,92.0,0.0


In [13]:
y.shape

(126391,)

In [14]:
from collections import Counter
counter = Counter(y)
for k, v in counter.items():
  dist = v/ len(y)*100
  print(f"Class={k}, n={v} ({dist}%)")

Class=0, n=51773 (40.96256853731674%)
Class=1, n=74618 (59.037431462683266%)


In [15]:
# Get list of categorical variables
data_cat = X_data.dtypes[X_data.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
X_data[data_cat].nunique()

course          67
band             8
race_group       4
race_type        2
trainer       2028
jockey        1650
gear            24
dtype: int64

In [16]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

enc_cats = enc.fit_transform(X_data[data_cat])

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc_cats)

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(data_cat)
encode_df.head()

,course_Aintree,course_Ascot,course_Ayr,course_Ballinrobe,course_Bangor-on-Dee,course_Bellewstown,course_Carlisle,course_Cartmel,course_Catterick,course_Cheltenham,...,gear_htp,gear_htv,gear_hv,gear_p,gear_t,gear_tb,gear_tp,gear_tv,gear_v,gear_vp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [17]:
len(encode_df)

126391

In [18]:
# Merge one-hot encoded features and drop the originals
#X_data = X_data.merge(encode_df, left_index=True, right_index=True)
#X_data = X_data.drop(data_cat, 1)
#X_data.head()

In [19]:
numerical_df = X_data.drop(columns=data_cat)
X_data = pd.concat([numerical_df,encode_df], axis=1)

In [20]:
len(X_data)

126391

In [22]:
len(y)

126391

#